### LangSmith Walkthrough

#### Resources
- [langsmith cookbook](https://github.com/langchain-ai/langsmith-cookbook/blob/main/tracing-examples/traceable/tracing_without_langchain.ipynb)

In [ ]:
!pip install langsmith langchain anthropic

In [ ]:
import langsmith, os, sys
import anthropic as anthropic_base
from langsmith import traceable
from pathlib import Path 

_root = "/home/zjc1002/Mounts/code/admin/"


sys.path.append(_root)
from api_keys import _api_keys


### Langsmith API Parameters

In [ ]:
!export LANGSMITH_TRACING_v2=true

In [ ]:
LANGSMITH_TRACING=True
LANGSMITH_ENDPOINT='https://api.smith.langchain.com'
LANGSMITH_PROJECT="LangSmith Basics"

LANGSMITH_API_KEY= _api_keys["LANGSMITH_API_KEY"]
ANTHROPIC_API_KEY=_api_keys['ANTHROPIC_API_KEY']
ANTHROPIC_MODEL='claude-3-haiku-20240307'

os.environ["LANGCHAIN_ENDPOINT"] = LANGSMITH_ENDPOINT
os.environ["LANGCHAIN_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGSMITH_PROJECT


### Create LLM INSTANCE
##### Note: I am using anthropic, but u should be able to use any model open source or not

In [ ]:
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

#### Trace a basic function

- **@traceable** decorator is ment to manage the logging of events from the client using a RunTree, which tracks your application. Each RunTree is required to have the below paramters
   - **name:str**: used to id the components purpose
   - **run_type:str**: 'llm','chain', or 'tool'. describes the type of interaction / activity being logged
   - **inputs:dict**: the inputs to the component
   - **outputs:Optional[dict]**: the (optinal) returned values from the component
   - **error: Optional[str]**: any error messages that may have arisin during the call


In [ ]:
@traceable # Auto-trace this function
def pipeline(client, user_input: str, model = None):

    assert model is not None, "Model must be specified, only ANTHROPIC models currently supported"
    result = client.messages.create(
        model= model
        , max_tokens = 1000
        , messages=[{"role": "user", "content": user_input}],

    )
    return result.content[0]

pipeline(client, "Hello, world!", model = ANTHROPIC_MODEL)